Nama : Virgie Yunita Salsabil

NIM : 21110022

Kelas : S1-SD02A

### **RESPONSI NLP #2**

Buatlah sistem/ aplikasi tanya jawab (QAS) dengan pilihan topik kesehatan sebagai berikut:

a. Kesehatan klinis

b. MBKM

Dataset bisa diunduh pada drive berikut: https://drive.google.com/drive/folders/1cgZC-_ORZugQarsfyji8bq0JkriLqp2I


**Bobot Penilaian:**
1. Text-Preprocessing (20 point)
2. Normalisasi/ vektorisasi (10 point)
3. Modeling ( 50 point)
4. Evaluasi model (10 point)
5. Uji/ testing dengan 10 pertanyaan yang Anda buat sendiri dan analisis hasilnya berapa
jawaban yang tepat atau relevan antara pertanyaan dan jawaban (10 point)

## Import Library

In [195]:
# Instalasi library Natural Language Toolkit (nltk) menggunakan pip
!pip install nltk

In [232]:
# Membaca file csv ke dalam DataFrame
import pandas as pd

# Library untuk operasi array numerik dan manipulasi data
import numpy as np

# Regular expression
import re

# Membagi teks menjadi token
from nltk.tokenize import word_tokenize

# Library untuk pemrosesan teks menggunakan TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Library untuk mengukur kemiripan kosinus antara dua set dokumen
from sklearn.metrics.pairwise import cosine_similarity

# Impor library NLTK
import nltk
from nltk.corpus import stopwords

# Download stopwords jika belum ada
nltk.download('stopwords')
nltk.download('punkt')

# Mendapatkan set stopwords untuk bahasa Indonesia
stop_words_id = set(stopwords.words('indonesian'))

# Menghapus stop words dari teks
from nltk.corpus import stopwords

# Import library untuk lemmatization
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [233]:
# Lihat Dataset
data = pd.read_csv("dataset_mentalhealth.csv")
data

,Question_ID,Questions,Jawaban
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident..."
...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba..."
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...


### **1. TEXT PREPROCESSING**

- Dengan langkah-langkah berikut, teks pada kolom 'Questions' dan 'Jawaban' akan bersih dari karakter non-huruf, angka, kata-kata tunggal, dan spasi yang tidak diinginkan.

In [234]:
# Fungsi preprocess untuk membersihkan teks
def text_prep(text):
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    # Menghapus karakter non-alfanumerik
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Menghapus spasi di awal teks
    text = re.sub(r'^\s+', '', text)
    return text

In [235]:
# Tokenisasi dan penghapusan stop words
def hapus_stopwords(text):
    stop_words = set(stopwords.words('indonesian'))
    words = text.split()
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return ' '.join(words)

In [236]:
# Clean & Tokenization "Question"
data['prep_question'] = data['Questions'].apply(text_prep)
data['prep_question'] = data['prep_question'].apply(hapus_stopwords)

In [237]:
# Clean & Tokenization "Jawaban"
data['prep_jawaban'] = data['Jawaban'].apply(text_prep)
data['prep_jawaban'] = data['prep_jawaban'].apply(hapus_stopwords)

In [238]:
# Lihat data setelah dilakukan preprocessing
data

,Question_ID,Questions,Jawaban,prep_question,prep_jawaban
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...,penyakit mental,penyakit mental kondisi kesehatan mengganggu p...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,terpengaruh penyakit mental,penyakit mental mempengaruhi 1 5 orang dewasa ...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,penyebab penyakit mental,penyakit mental mempengaruhi 1 5 orang dewasa ...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...,sajakah tandatanda peringatan penyakit mental,gejala gangguan kesehatan mental bervariasi te...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident...",penderita penyakit jiwa sembuh,penyembuhan penyakit mental identifikasi pengo...
...,...,...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...,minum,menyortir minum rumit unik hubungan alkohol un...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba...",ganja berbahaya melegalkannya,asap ganja mengandung racun penyebab kanker ri...
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...,anak narkoba,mempengaruhi kapasitas pilihan dunia alkohol o...
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...,status hukum bukti minyak cbd,cannabidiol cbd komponen ganja alami diekstrak...


In [239]:
# Gabungkan teks dari kolom Question dan Jawaban
data['text_combined'] = data['prep_question'] + ' ' + data['prep_jawaban']

# Tampilan Data setelah digabungkan
data

,Question_ID,Questions,Jawaban,prep_question,prep_jawaban,text_combined
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...,penyakit mental,penyakit mental kondisi kesehatan mengganggu p...,penyakit mental penyakit mental kondisi keseha...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,terpengaruh penyakit mental,penyakit mental mempengaruhi 1 5 orang dewasa ...,terpengaruh penyakit mental penyakit mental me...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,penyebab penyakit mental,penyakit mental mempengaruhi 1 5 orang dewasa ...,penyebab penyakit mental penyakit mental mempe...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...,sajakah tandatanda peringatan penyakit mental,gejala gangguan kesehatan mental bervariasi te...,sajakah tandatanda peringatan penyakit mental ...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident...",penderita penyakit jiwa sembuh,penyembuhan penyakit mental identifikasi pengo...,penderita penyakit jiwa sembuh penyembuhan pen...
...,...,...,...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...,minum,menyortir minum rumit unik hubungan alkohol un...,minum menyortir minum rumit unik hubungan alko...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba...",ganja berbahaya melegalkannya,asap ganja mengandung racun penyebab kanker ri...,ganja berbahaya melegalkannya asap ganja menga...
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...,anak narkoba,mempengaruhi kapasitas pilihan dunia alkohol o...,anak narkoba mempengaruhi kapasitas pilihan du...
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...,status hukum bukti minyak cbd,cannabidiol cbd komponen ganja alami diekstrak...,status hukum bukti minyak cbd cannabidiol cbd ...


In [240]:
# Simpan data dalam bentuk Excel
data.to_excel("Hasil Preprocessing.xlsx")

### **2. VEKTORISASI**

- TF-IDF

In [256]:
# Membuat vektor TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Menghitung nilai TF-IDF dari teks
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text_combined'])
tfidf_matrix

<98x2107 sparse matrix of type '<class 'numpy.float64'>'
	with 7296 stored elements in Compressed Sparse Row format>

### **3. MODELLING**

In [257]:
# Fungsi untuk mendapatkan jawaban berdasarkan pertanyaan menggunakan cosine similarity
def answering(question, tfidf_matrix):

    # Menghasilkan vektor TF-IDF untuk pertanyaan
    question_vector = tfidf_vectorizer.transform([question])

    # Menghitung cosine similarity antara vektor pertanyaan dan vektor jawaban
    similarities = cosine_similarity(question_vector, tfidf_matrix).flatten()

    # Mendapatkan indeks jawaban dengan similarity tertinggi
    answer_index = similarities.argmax()

    # Mengembalikan jawaban dari indeks yang ditemukan
    return data['Jawaban'][answer_index]

In [279]:
# PENGGUNAAN MODEL
pertanyaan = "Apa yang dimaksud dengan penyakit mental?"
jawaban = answering(pertanyaan, tfidf_matrix)

print("Pertanyaan: ", pertanyaan)
print("-"*50)
print("Jawaban: \n", jawaban)

Pertanyaan:  Apa yang dimaksud dengan penyakit mental?
--------------------------------------------------
Jawaban: 
 'Kesehatan mental' dan 'penyakit mental' semakin banyak digunakan seolah -olah memaksudkan hal yang sama, tetapi tidak. Setiap orang memiliki kesehatan mental, sama seperti semua orang memiliki kesehatan. Seperti yang dikatakan Organisasi Kesehatan Dunia dengan terkenal, “Tidak ada kesehatan tanpa kesehatan mental.” Dalam perjalanan seumur hidup, tidak semua orang akan mengalami penyakit mental, tetapi semua orang akan berjuang atau memiliki tantangan dengan kesejahteraan mental mereka (mis., Kesehatan mental mereka) sama seperti kita semua memiliki tantangan dengan kesejahteraan fisik kita dari waktu demi waktu.
 Ketika kita berbicara tentang kesehatan mental, kita berbicara tentang kesejahteraan mental kita: emosi kita, pikiran dan perasaan kita, kemampuan kita untuk menyelesaikan masalah dan mengatasi kesulitan, koneksi sosial kita, dan pemahaman kita tentang dunia di

### **5. UJI 10 PERTANYAAN**

In [276]:
# List Pertanyaan
Uji_Pertanyaan = [
    "Bagaimana kondisi kesehatan mental dapat mengganggu pikiran, emosi, dan hubungan?",
    "Apa yang dapat menyebabkan terjadinya penyakit mental?",
    "Apa saja tanda-tanda peringatan penyakit mental, terutama gejala gangguan kesehatan mental yang parah?",
    "Bagaimana cara mengatasi stres dalam konteks kesehatan mental?",
    "Apakah penyakit mental dapat sembuh, dan apa yang diperlukan untuk mencapai pemulihan?",
    "Bagaimana penyakit mental memengaruhi fungsionalitas utama dan kehidupan sehari-hari seseorang?",
    "Apa peran perawatan rumah sakit dalam mengatasi penyakit mental serius seperti depresi berat atau skizofrenia?",
    "Bagaimana identifikasi dan pengobatan gangguan mental sulit, terutama pada remaja yang mengalami perubahan perilaku?",
    "Apa saja metode dukungan sosial dan perawatan efektif yang dapat membantu individu dengan penyakit mental?",
    "Apa peran keluarga dan pengasuh dalam mendukung individu yang mengalami gangguan kesehatan mental?"]

# Fungsi Uji dan Analisis hasil
def performa(questions, tfidf_matrix):
    # Inisialisasi list untuk menyimpan hasil prediksi
    results = []

    # Iterasi melalui setiap pertanyaan pada data uji
    for idx, Uji_Pertanyaan in enumerate(questions):
        # Mendapatkan jawaban prediksi menggunakan fungsi answering
        predicted_answer = answering(Uji_Pertanyaan, tfidf_matrix)

        # Menyusun hasil prediksi dalam bentuk dictionary
        result_dict = {
            "Question": Uji_Pertanyaan,
            "Prediction Answear": predicted_answer}

        # Menambahkan hasil prediksi ke dalam list
        results.append(result_dict)

        # Menampilkan pertanyaan, jawaban prediksi, dan baris baru untuk pemisah
        print(f"Question {idx + 1}: {Uji_Pertanyaan}")
        print("-"*100)
        print(f"Prediction Answear: {predicted_answer}\n")
        print("-"*100)

    # Mengembalikan hasil prediksi dalam bentuk DataFrame
    return pd.DataFrame(results)

# Simpan hasil uji ke Excel
results_test = performa(Uji_Pertanyaan, tfidf_matrix)
results_test.to_excel("Hasil Uji - Chatbot Mental Health.xlsx.xlsx", index=False)

Question 1: Bagaimana kondisi kesehatan mental dapat mengganggu pikiran, emosi, dan hubungan?
----------------------------------------------------------------------------------------------------
Prediction Answear: Penyakit mental adalah kondisi kesehatan yang mengganggu pikiran, emosi, hubungan, dan fungsi seseorang. Mereka dikaitkan dengan kesusahan dan berkurangnya kapasitas untuk terlibat dalam kegiatan kehidupan sehari -hari yang biasa.
Penyakit mental jatuh di sepanjang rangkaian keparahan: beberapa cukup ringan dan hanya mengganggu beberapa aspek kehidupan, seperti fobia tertentu. Di ujung lain spektrum terletak penyakit mental yang serius, yang mengakibatkan gangguan fungsional utama dan gangguan dengan kehidupan sehari -hari. Ini termasuk gangguan seperti depresi berat, skizofrenia, dan gangguan bipolar, dan mungkin mengharuskan orang tersebut menerima perawatan di rumah sakit.
Penting untuk mengetahui bahwa penyakit mental adalah kondisi medis yang tidak ada hubungannya denga

### **Evaluasi Model**

Evaluasi Model Berdasarkan 10 Pertanyaan:

Akurasi Jawaban:
- Model memberikan jawaban yang cukup informatif dan relevan terhadap pertanyaan-pertanyaan yang diajukan.
- Jawaban model mencakup definisi, penjelasan, dan informasi tambahan yang relevan.

Konsistensi Jawaban:
- Jawaban model cenderung konsisten dalam memberikan penjelasan umum mengenai topik kesehatan mental.
- Informasi yang disajikan terkadang berulang, seperti penekanan bahwa penyakit mental adalah kondisi medis.

Ketepatan Informasi:
- Informasi yang disajikan oleh model terlihat cukup akurat dan sesuai dengan pengetahuan umum tentang kesehatan mental.
- Terdapat beberapa penjelasan tentang istilah medis, seperti "prodrome," yang dapat membantu dalam pemahaman lebih lanjut.

Ketepatan Konteks:
- Jawaban model terkadang cenderung bersifat deskriptif dan kurang fokus pada konteks yang lebih spesifik.
- Beberapa jawaban memberikan informasi umum yang mungkin dapat diperdalam dengan konteks lebih lanjut.

Kekayaan Jawaban:
- Model memberikan jawaban yang cukup kaya dan terperinci, terutama pada pertanyaan yang memerlukan penjelasan lebih mendalam, seperti pertanyaan 3 dan 4.

## **KESIMPULAN**

Kelebihan Model:
- Model memberikan jawaban yang informatif dan mendalam terkait topik kesehatan mental.
- Pemahaman umum tentang kondisi kesehatan mental disampaikan dengan baik.

Kelemahan Model:
- Beberapa jawaban terlihat terlalu umum dan kurang fokus pada konteks yang lebih spesifik.
- Ada potensi peningkatan dalam konsistensi dan variasi jawaban untuk pertanyaan yang serupa.

Rekomendasi untuk Peningkatan:
- Model dapat diperbarui dengan lebih mempertimbangkan konteks pertanyaan dan memberikan jawaban yang lebih terfokus.
- Evaluasi lebih lanjut dapat dilakukan dengan melibatkan lebih banyak sumber dan perbanyak dataset terkait kesehatan mental.
- Pembaruan terkait konten dan struktur jawaban dapat meningkatkan presisi dan ketepatan model.